In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import chaospy
import pathlib
import warnings

#from statistics import NormalDist
from pathlib import Path  
from scipy.stats import t

# Local packages
import optionpricing
from optionpricing import OptionPricingClass

# Settings
%load_ext autoreload
%autoreload 2

# Parent folder
pf = pathlib.Path().resolve() # Points to parent folder containing notebook and data, eg: 'C:/Users/Carla/Dropbox/Uni/10. Semester/Dynamic Programming/Term paper'

# Plots & Tables

## Section: Replication

### Comparison of simulation results to the results of Longstaff and Schwartz

In [2]:
### for testing you can vary the number of sims here # it takes roughly 1 minute with 10k sims, about 10 minutes with 100k sims (100k are used the paper)
simulations = 100000
###
t1 = time.time()
# initial parameters
S0, strike, T, M, r, sigma, deg, basis_func = 36, 40, 1, 50, 0.06, 0.2, 5, 'laguerre' 
reference_value = None
halton = None

# for saving results
initial_prices, sigmas, Ts, prices, times, std_error = [[],[],[],[],[],[]]

method = 'monte carlo'

# run simulation - loop over parameters  
for S0 in (36, 38, 40, 42, 44):  # initial stock price values
    for sigma in (0.2, 0.4):  # voltaility
        for T in (1.0, 2.0):  # years
            # run sim
            price, time_elapsed, standard_dev, standard_error, relative_error = optionpricing.run_simulations(S0, strike, T, M, r, sigma, simulations, deg, basis_func, method, halton, reference_value)

            # save results
            initial_prices.append(S0), sigmas.append(sigma), Ts.append(T), prices.append(price),
            times.append(time_elapsed), std_error.append(standard_error)       

# format and save results to csv so we dont have to rerun sims
df = pd.DataFrame(list(zip(initial_prices, sigmas, Ts, prices, std_error, times)),
              columns=['S0','sigma', 'T', 'price', 's.e','time elapsed'])
filepath = Path(f'{pf}/data/Table2_{basis_func}_{simulations}.csv') # Set name 
filepath.parent.mkdir(parents=True, exist_ok=True) 
df.to_csv(filepath, index=False)
print(f'simulation took {(time.time()-t1)/60:1.3f} minutes ')
df

C:\Users\Carla\anaconda3\lib\site-packages\numpy\polynomial\laguerre.py:1401: RankWarning: The fit may be poorly conditioned
  return pu._fit(lagvander, x, y, deg, rcond, full, w)


simulation took 0.088 minutes 


,S0,sigma,T,price,s.e,time elapsed
0,36,0.2,1.0,4.572968,0.097126,0.321026
1,36,0.2,2.0,4.878353,0.102069,0.317114
2,36,0.4,1.0,7.361634,0.183231,0.242359
3,36,0.4,2.0,8.753986,0.217782,0.276674
4,38,0.2,1.0,3.320477,0.090546,0.261033
5,38,0.2,2.0,3.877922,0.108608,0.257369
6,38,0.4,1.0,6.386353,0.176398,0.303354
7,38,0.4,2.0,7.911257,0.207884,0.249544
8,40,0.2,1.0,2.373957,0.088846,0.249904
9,40,0.2,2.0,3.008073,0.106890,0.246562


### Choice of polynomial

In [8]:
### for testing you can vary the number of sims here
simulations = 10000
###
method = 'monte carlo'

# When the number of simulations gets low, np will return a warning. This is ignored
warnings.simplefilter('ignore', np.RankWarning)

# initial parameters
S0, strike, T, M, r, sigma, deg, basis_func = 36, 40, 1, 50, 0.06, 0.2, 5, 'laguerre' 


# methods to loop over
basis_functions = ['polyfit', 'chebyshev', 'laguerre', 'hermite', 'legendre'] #'manual'

# for saving results
basis_m, deg_m, price_m, time_m = [[],[],[],[]]

for i, basis_func in enumerate(basis_functions):
    basis_m, deg_m, price_m, time_m = [[],[],[],[]]
    for j in range(2,10):
        # start timer
        t0 = time.time() 
        # initialize class 
        PUT = OptionPricingClass(S0, strike, T, M, r, sigma,simulations, j, basis_func, method)
        # price option
        price, standard_dev, standard_error = PUT.price
        # end timer
        t1 = time.time()
        time_elapsed = t1-t0

        basis_m.append(basis_func), deg_m.append(j), price_m.append(price), time_m.append(time_elapsed)
    if basis_func == 'polyfit':
        results = pd.DataFrame((zip(basis_m, deg_m, price_m)), columns=['Basis','degree', 'price'])
    else:
        results_1 = pd.DataFrame((zip(basis_m, deg_m, price_m)), columns=['Basis','degree', 'price'])
        results = pd.merge(left=results, right=results_1, left_on='degree', right_on='degree')
results.drop(['Basis_x', 'Basis_y', 'Basis_x','Basis_y','Basis'], axis=1, inplace=True)
results.columns = ['degree', 'polyfit', 'chebyshev', 'laguerre', 'hermite', 'legendre']
results.set_index('degree', inplace=True)
results

C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\chebyshev.py:1671: RankWarning: The fit may be poorly conditioned
  return pu._fit(chebvander, x, y, deg, rcond, full, w)
C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\laguerre.py:1401: RankWarning: The fit may be poorly conditioned
  return pu._fit(lagvander, x, y, deg, rcond, full, w)
C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\hermite.py:1404: RankWarning: The fit may be poorly conditioned
  return pu._fit(hermvander, x, y, deg, rcond, full, w)
C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\legendre.py:1412: RankWarning: The fit may be poorly conditioned
  return pu._fit(legvander, x, y, deg, rcond, full, w)


,polyfit,chebyshev,laguerre,hermite,legendre
degree,,,,,
2,4.490344,4.490344,4.490344,4.490344,4.490344
3,4.503774,4.503774,4.503774,4.503774,4.503774
4,4.512785,4.512785,4.512785,4.512785,4.512785
5,4.509402,4.509402,4.509402,4.509402,4.509402
6,4.503963,4.503963,4.503963,4.503963,4.503963
7,4.497098,4.497098,4.497098,4.497098,4.497098
8,4.499305,4.499305,4.499749,4.499305,4.499305
9,4.502576,4.502576,4.502693,4.496894,4.502576


## Section: Least Squares Monte Carlo

In [ ]:
method = 'monte carlo'

M_range = range(10, 210, 20) # it will also be much faster with a low number of time steps and few iterables

basis_functions = ['polyfit', 'chebyshev', 'laguerre', 'hermite', 'legendre'] #'manual'
basis_func = 'laguerre'
deg = 5

# fixed parameters - true option values are based on these
reference_values = (10.726486710094511, 4.820608184813253, 1.828207584020458) # TRUE OPTION VALUES
S0s = (90, 100, 110) 
strike = 100
T = 1
r = 0.03
sigma = 0.15

# for saving results
initial_prices, sigmas, Ms, prices, times, std_error, std_dev, rel_error, basis_funcs, degs, sims = [[],[],[],[],[],[],[],[],[],[],[]]

# run simulations
t1_ = time.time()
# loop over parameters  
for simulations in (100, 1000, 10000): 
    for i, S0 in enumerate(S0s):  
        for deg in (50, 20, 10, 5, 4, 3, 2):
            for M in M_range:  
                # run sim
                price, time_elapsed, standard_dev, standard_error, relative_error = optionpricing.run_simulations(S0, strike, T, M, r, sigma, simulations, deg, basis_func, method, halton, reference_values[i])

                # save results
                initial_prices.append(S0), sigmas.append(sigma), Ms.append(M), prices.append(price), basis_funcs.append(basis_func), degs.append(deg), times.append(time_elapsed),
                std_dev.append(standard_dev), std_error.append(standard_error), rel_error.append(relative_error), sims.append(simulations)

    # format and save results to csv so we dont have to rerun sims
    df = pd.DataFrame(list(zip(initial_prices, Ms, prices, times, std_error, rel_error, basis_funcs, degs, sims)),
                  columns=['S0', 'M', 'price', 'time elapsed', 's.e', 'rel. error', 'basis func', 'degrees','simulations'])
    filepath = Path(f'{pf}/data/accuracy_{basis_func}_{len(M_range)}Ms_{simulations}sims.csv') # Set name 
    df.to_csv(filepath, index=False)
    
t2_ = time.time()
print(f'simulation took {(t2_-t1_)/60:1.3f} minutes ')
df.head()

C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\laguerre.py:1401: RankWarning: The fit may be poorly conditioned
  return pu._fit(lagvander, x, y, deg, rcond, full, w)


## Brownian bridge

In [14]:
method = 'brownian bridge'

# fixed parameters
reference_values = (10.726486710094511, 4.820608184813253, 1.828207584020458) # TRUE OPTION VALUES
strike = 100
T = 1
r = 0.03
sigma = 0.15
S0s = (90, 100, 110) # vary between these 3
deg = 5

# simulation settings
M_range = range(10, 210, 20) 
#simulations = 10000

# for saving results
initial_prices, sigmas, Ms, prices, times, std_error, std_dev, rel_error, basis_funcs, degs, sims, haltons = [[],[],[],[],[],[],[],[],[],[],[],[]]

# loop over parameters
for simulations in (1000, 10000, 100000, 250000):
    t1_ = time.time()
    for halton in (True, False):
        for i, S0 in enumerate(S0s):  # initial stock price values
            for M in M_range:  # sigma
                # run sim
                price, time_elapsed, standard_dev, standard_error, relative_error = optionpricing.run_simulations(S0, strike, T, M, r, sigma, simulations, deg, basis_func, method, halton, reference_values[i])

                # save results
                initial_prices.append(S0), sigmas.append(sigma), Ms.append(M), prices.append(price), basis_funcs.append(basis_func), degs.append(deg), times.append(time_elapsed),
                std_dev.append(standard_dev), std_error.append(standard_error), rel_error.append(relative_error), sims.append(simulations), haltons.append(halton)

    # format and save results to csv so we dont have to rerun sims
    df = pd.DataFrame(list(zip(initial_prices, Ms, prices, times, std_error, rel_error, basis_funcs, degs, sims, haltons)),
                  columns=['S0', 'M', 'price', 'time elapsed', 's.e', 'rel. error', 'basis func', 'degrees','simulations', 'halton'])
    filepath = Path(f'{pf}/data/accuracy_BB_{basis_func}_{len(M_range)}Ms_{simulations}sims.csv') # Set name 
    df.to_csv(filepath, index=False)
    t2_ = time.time()
    total_time=(t2_-t1_)/60
    print(f'{simulations} simulation took {total_time:1.3f} minutes)')
df

C:\Programs\Anaconda3\lib\site-packages\numpy\polynomial\laguerre.py:1401: RankWarning: The fit may be poorly conditioned
  return pu._fit(lagvander, x, y, deg, rcond, full, w)


1000 simulation took 0.038 minutes)
10000 simulation took 0.139 minutes)
100000 simulation took 1.754 minutes)
250000 simulation took 6.000 minutes)


,S0,M,price,time elapsed,s.e,rel. error,basis func,degrees,simulations,halton
0,90,10,10.657040,0.014107,0.210479,0.004823,laguerre,5,1000,True
1,90,30,10.750572,0.013376,0.207998,0.000580,laguerre,5,1000,True
2,90,50,10.887865,0.021435,0.216574,0.026043,laguerre,5,1000,True
3,90,70,10.602620,0.018960,0.161002,0.015343,laguerre,5,1000,True
4,90,90,10.741684,0.046444,0.174725,0.000231,laguerre,5,1000,True
...,...,...,...,...,...,...,...,...,...,...
235,110,110,1.820737,2.758669,0.007571,0.000056,laguerre,5,250000,False
236,110,130,1.818623,3.311521,0.007567,0.000092,laguerre,5,250000,False
237,110,150,1.818957,3.919391,0.007543,0.000086,laguerre,5,250000,False
238,110,170,1.819911,4.390759,0.007542,0.000069,laguerre,5,250000,False
